In [22]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import itertools

## **Implementación**

In [83]:
#variables 
x = np.array([[-1,-2]]).T
y = np.array([[0,1]]).T

def calc_backpropagation(x,y,act_func = None, floss = None):
  #parámetros 2 neuronas y 2 datos de la variable dependiente
  W = np.array([[2,-3]])
  #En caso de querer pesos sinápticos random: W = np.random.randn(3,4) * 0.01 
  b = np.zeros((2,1))
  # de los siguientes resultados el indice del parametro representa el indice respectivo a la neurona
  print("PARÁMETROS ORIGINALES\npesos sinápticos:\n{} \nsesgo:\n{}".format(W,b))

  z = W@x + b
  
  #--------------------------------Forward pass-----------------------------
  #función de activación predeterminada sigmoidal
  def activation_function(x,f = None):
    if f == "softmax": 
      exp_x = np.exp(x)
      sum_exp = np.sum(exp_x)
      return exp_x / sum_exp
    elif f == "tanh":
      exp_x = np.exp(x)
      neg_exp_x = np.exp(-x)
      return (exp_x - neg_exp_x) / (exp_x + neg_exp_x)
    else:
      exp_x = np.exp(-x)
      return 1/(1+exp_x)    

  y_hat = activation_function(z,act_func)
  
  #-----------------función de pérdida 
  # por default Cross-entropy 
  #se obtiene solo para donde se marca la clase correcta
  def loss_function(y0,ypred, f = None):
    if f == "mse":
      return sum((y0-ypred)[0])**2
    else:
      return -np.log(y_hat[np.where(y==1)])

  loss_i = loss_function(y, y_hat)
  print('loss function inicial: {}'.format(loss_i))
  #------------------------------Backpropagation----------------------------
  #Gradiente dloss_i = 1
  def backprop(f = None):
    if f == "softmax": #solo se puede ocupar para función de perdida xentropy
      dz = y_hat - y
    elif f == "tanh":
      dz = 1 - y_hat ** 2
    else:
      dz = (np.exp(1))/((1+np.exp(1))**2) #Gradiente Sigmoid
    
    dW = dz * x.T #Gradiente pesos sinápticos
    db = dz #Gradiente sesgo
    return dW, db
  
  dW, db = backprop(act_func)
    
  #---------------------------Actualizar parámetros-------------------------
  learning_rate = 1e-2
  W = W - learning_rate * dW
  b = b - learning_rate * db

  print("\nPARÁMETROS FINALES\n pesos sinápticos:\n{} \nsesgo:\n{} ".format(W,b))
  

  z = (W@x + b)
  y_hat = activation_function(z,act_func)
  print('loss function inicial: {}'.format(loss_function(y, y_hat)))
calc_backpropagation(x,y)

PARÁMETROS ORIGINALES
pesos sinápticos:
[[ 2 -3]] 
sesgo:
[[0.]
 [0.]]
loss function inicial: [0.01814993]

PARÁMETROS FINALES
 pesos sinápticos:
[[ 2.00196612 -2.99606776]] 
sesgo:
[[-0.00196612]
 [-0.00196612]] 
loss function inicial: [0.01836334]


## **Validación**

**Micrograd**

In [ ]:
!pip install micrograd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
from micrograd.nn import MLP

xs = [-1.0, -2.0]
ys = [0.0, 1.0]

n = MLP(2, [2, 2]) #La función de activación que ocupa la API es tanh

for k in range(2):
  #--------------------------Forward pass---------------------------
  ypred = n(x)

  # al no poderse aplicar una funcion log en micrograd se obtiene mse
  loss = sum((yout - ygt) ** 2 for ygt, yout in zip(ys, ypred))

  #-------------------------Backward pass--------------------------
  for p in n.parameters():
    p.grad = 0.0
  loss.backward()

  # update
  for p in n.parameters():
    p.data += 0.1 * p.grad

  print('iteración #{}'.format(k), '\nloss function: {}\n'.format(loss.data), '\nparametros: {}'.format(n.parameters()))

iteración #0 
loss function: [0.33284065]
 
parametros: [Value(data=[0.30302852], grad=[0.]), Value(data=[0.05507779], grad=[0.]), Value(data=[0.], grad=[0.]), Value(data=[-0.79760775], grad=[0.03202069]), Value(data=[-0.15947872], grad=[0.06404139]), Value(data=[-0.00320207], grad=[-0.03202069]), Value(data=[-0.42907448], grad=[0.]), Value(data=[0.50800379], grad=[1.03717753]), Value(data=[0.0915769], grad=[0.91576896]), Value(data=[0.80154539], grad=[0.]), Value(data=[0.49355272], grad=[-0.79500818]), Value(data=[-0.07019472], grad=[-0.70194716])]
iteración #1 
loss function: [0.70299132]
 
parametros: [Value(data=[0.30302852], grad=[0.]), Value(data=[0.05507779], grad=[0.]), Value(data=[0.], grad=[0.]), Value(data=[-0.81297896], grad=[-0.15371206]), Value(data=[-0.19022113], grad=[-0.30742412]), Value(data=[0.01216914], grad=[0.15371206]), Value(data=[-0.42907448], grad=[0.]), Value(data=[0.65433747], grad=[1.46333677]), Value(data=[0.22301081], grad=[1.31433918]), Value(data=[0.801

In [86]:
calc_backpropagation(x,y, "tanh", "mse")

PARÁMETROS ORIGINALES
pesos sinápticos:
[[ 2 -3]] 
sesgo:
[[0.]
 [0.]]
loss function inicial: [0.00067093]

PARÁMETROS FINALES
 pesos sinápticos:
[[ 2.00001341 -2.99997318]
 [ 2.00001341 -2.99997318]] 
sesgo:
[[-1.34095068e-05]
 [-1.34095068e-05]] 
loss function inicial: [0.00067103]


### **Tinygrad**

In [ ]:
#! git clone https://github.com/geohot/tinygrad.git
#%cd tinygrad

/content/tinygrad


In [ ]:
! python3 setup.py develop

running develop
running egg_info
writing tinygrad.egg-info/PKG-INFO
writing dependency_links to tinygrad.egg-info/dependency_links.txt
writing requirements to tinygrad.egg-info/requires.txt
writing top-level names to tinygrad.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'tinygrad.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/tinygrad.egg-link (link to .)
tinygrad 0.4.0 is already the active version in easy-install.pth

Installed /content/tinygrad
Processing dependencies for tinygrad==0.4.0
Searching for networkx==2.6.3
Best match: networkx 2.6.3
Adding networkx 2.6.3 to easy-install.pth file

Using /usr/local/lib/python3.7/dist-packages
Searching for Pillow==7.1.2
Best match: Pillow 7.1.2
Adding Pillow 7.1.2 to easy-install.pth file

Using /usr/local/lib/python3.7/dist-packages
Searching for requests==2.23.0
Best match: requests 2.23.0
Adding requests 2.23.0 to easy-install.pth file

Using /usr/local/lib/python3.7/